<a href="https://colab.research.google.com/github/CMWENLIU/Drug_label_embedding/blob/master/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install --quiet word2vec
! pip install --quiet wget

In [0]:
import wget
import word2vec
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

# word2vec 

This notebook is equivalent to `demo-word.sh`, `demo-analogy.sh`, `demo-phrases.sh` and `demo-classes.sh` from Google.

In [0]:
%load_ext autoreload
%autoreload 2
urls = ["https://raw.githubusercontent.com/CMWENLIU/Drug_label_embedding/master/pubmed/abs1.txt",
       "https://raw.githubusercontent.com/CMWENLIU/Drug_label_embedding/master/pubmed/abs2.txt",
       ]
[wget.download(u) for u in urls]   

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


['abs1 (2).txt', 'abs2 (2).txt']

In [0]:
def clean_str(string):
   
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = string.replace("\\","")
    return string.strip().lower()

def split_record(s):
  journal = s.split('-!!-', 1)[0]
  temp = s.split('-!!-', 1)[1]
  year = temp.split('-##-', 1)[0]
  abstract = s.split('-##-', 1)[1]
  return journal, year, abstract

In [0]:
#with open('w2vdata', 'w') as wf:
#  with open('abs2.txt') as f:
#    for line in f:
#      if '-##-' in line:
#        line = clean_str(line.split('-##-', 1)[1])
#        wf.write(line)

In [0]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 
with open('w2vdata', 'w') as wf:
  with open('TS.txt') as f:
    for line in f:
        line = clean_str(line)
        #line = [w for w in line if not w in stop_words] 
        wf.write(line)

## Training

Run `word2phrase` to group up similar words "Los Angeles" to "Los_Angeles"

In [0]:
word2vec.word2phrase('w2vdata', 'w2vdata-phrases', verbose=True)

Starting training using file w2vdata

Vocab size (unigrams + bigrams): 481486
Words in train file: 2130459


This created a `text8-phrases` file that we can use as a better input for `word2vec`.
Note that you could easily skip this previous step and use the text data as input for `word2vec` directly.

Now actually train the word2vec model.

In [0]:
word2vec.word2vec('w2vdata-phrases', 'w2vdata.bin', size=100, verbose=True)

Starting training using file w2vdata-phrases
Vocab size: 25079
Words in train file: 1758403
Alpha: 0.000002  Progress: 100.42%  Words/thread/sec: 187.17k  

That created a `text8.bin` file containing the word vectors in a binary format.

Now we generate the clusters of the vectors based on the trained model.

In [0]:
word2vec.word2clusters('w2vdata', 'w2vdata-clusters.txt', 100, verbose=True)

Starting training using file w2vdata
Vocab size: 20800
Words in train file: 1941197
Alpha: 0.000002  Progress: 100.41%  Words/thread/sec: 188.30k  

That created a `text8-clusters.txt` with the cluster for every word in the vocabulary

## Predictions

In [0]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Import the `word2vec` binary file created above

In [0]:
model = word2vec.load('w2vdata.bin')

We can take a look at the vocabulary as a numpy array

In [0]:
model.vocab

array(['</s>', ',', 'of', ..., 'bmo', 'bbo', 'sorbents'], dtype='<U78')

Or take a look at the whole matrix

In [0]:
model.vectors.shape

(25079, 100)

In [0]:
model.vectors

array([[ 0.14333282,  0.15825513, -0.13715845, ...,  0.05456942,
         0.10955409,  0.00693387],
       [-0.10035713,  0.05325497,  0.06101317, ...,  0.16725662,
        -0.0775808 , -0.04770358],
       [-0.14344479, -0.0397615 ,  0.11518496, ..., -0.0714288 ,
         0.02132738,  0.12340144],
       ...,
       [-0.1776311 , -0.09941112, -0.14281759, ...,  0.1621709 ,
        -0.00876815, -0.07769725],
       [-0.09071511, -0.07830397,  0.04456994, ...,  0.13834722,
         0.13762638, -0.1301478 ],
       [-0.20501305, -0.10979778, -0.00982187, ...,  0.10732359,
        -0.06530824, -0.04285794]])

We can retreive the vector of individual words

In [0]:
model['cell'].shape

(100,)

In [0]:
model['cell'][:10]

array([-0.02201957,  0.13889974,  0.01483452, -0.14323322, -0.02133687,
        0.03510084, -0.10194293,  0.08460193, -0.03827839, -0.06916734])

We can calculate the distance between two or more (all combinations) words.

In [0]:
model.distance("egg", "protein", "gluten", "tumor","cancer", "disease", "analytic", "scientific", "diagnostic")

KeyError: ignored

## Similarity

We can do simple queries to retreive words similar to "socks" based on cosine similarity:

In [0]:
indexes, metrics = model.similar("cell")
indexes, metrics

(array([  306,   591,   480,   880, 13796,   983,   602,    33,   711,
         1156]),
 array([0.75267106, 0.74565173, 0.74452586, 0.73334029, 0.70504718,
        0.70163644, 0.70144433, 0.69382283, 0.68474115, 0.68359934]))

This returned a tuple with 2 items:
1. numpy array with the indexes of the similar words in the vocabulary
2. numpy array with cosine similarity to each word

We can get the words for those indexes

In [0]:
model.vocab[indexes]

array(['primary', 'epithelial', 'derived', 'murine', 'cd4_kj',
       'hepatocyte', 'cell_line', 'cells', 'cultures', 'bone_marrow'],
      dtype='<U78')

There is a helper function to create a combined response as a numpy [record array](http://docs.scipy.org/doc/numpy/user/basics.rec.html)

In [0]:
model.generate_response(indexes, metrics)

rec.array([('animal', 0.94223076), ('experimental', 0.8944898 ),
           ('preclinical', 0.87991323), ('toxicogenomics', 0.85871615),
           ('toxicologyresearch', 0.85729209), ('food', 0.85487465),
           ('epidemiology', 0.83767078), ('environmentaland', 0.83595806),
           ('toxic_substances', 0.83162512), ('industrial', 0.82925473)],
          dtype=[('word', '<U78'), ('metric', '<f8')])

Is easy to make that numpy array a pure python response:

In [0]:
model.generate_response(indexes, metrics).tolist()

[('animal', 0.9422307571217683),
 ('experimental', 0.8944898013342577),
 ('preclinical', 0.879913234534419),
 ('toxicogenomics', 0.8587161536414003),
 ('toxicologyresearch', 0.8572920856200303),
 ('food', 0.8548746496140631),
 ('epidemiology', 0.8376707770763547),
 ('environmentaland', 0.8359580588585981),
 ('toxic_substances', 0.831625124038414),
 ('industrial', 0.829254732607843)]

### Phrases

Since we trained the model with the output of `word2phrase` we can ask for similarity of "phrases", basically compained words such as "Los Angeles"

In [0]:
indexes, metrics = model.similar('cell')
model.generate_response(indexes, metrics).tolist()

[('primary', 0.75267106249543),
 ('epithelial', 0.7456517271791423),
 ('derived', 0.7445258619401021),
 ('murine', 0.7333402941904732),
 ('cd4_kj', 0.7050471750765015),
 ('hepatocyte', 0.701636436826913),
 ('cell_line', 0.701444331042083),
 ('cells', 0.69382283358038),
 ('cultures', 0.6847411456441168),
 ('bone_marrow', 0.6835993449894815)]

### Analogies

Its possible to do more complex queries like analogies such as: `king - man + woman = queen` 
This method returns the same as `cosine` the indexes of the words in the vocab and the metric

In [0]:
indexes, metrics = model.analogy(pos=['king', 'woman'], neg=['man'])
indexes, metrics

In [0]:
model.generate_response(indexes, metrics).tolist()

### Clusters

In [0]:
clusters = word2vec.load_clusters('/Users/drodriguez/Downloads/text8-clusters.txt')

We can see get the cluster number for individual words

In [0]:
clusters.vocab

We can see get all the words grouped on an specific cluster

In [0]:
clusters.get_words_on_cluster(90).shape

In [0]:
clusters.get_words_on_cluster(90)[:10]

We can add the clusters to the word2vec model and generate a response that includes the clusters

In [0]:
model.clusters = clusters

In [0]:
indexes, metrics = model.analogy(pos=["paris", "germany"], neg=["france"])

In [0]:
model.generate_response(indexes, metrics).tolist()